# Baseball Pitch Type Prediction

    Living in Houston, it would be sacreligious for me to not be a huge fan of baseball and the Hosuton Astros. For years, I've cheered, cried, and laughed as the Astros went from a pretty bad team to arguably the best team in the league (no matter what the Yankees say!). We acquired some of the best batters in the league as well as one of the greatest pitching lineups in MLB history. One of these incredibe pitchers is Houston's favorite closer, Ryan Pressly. He has come to be known as the ninth inning pitcher who never lets up a run, which is why I want to attempt to make a predictive algorithm that can determine what pitch Pressly would throw based on the status of the game at the time (ie. the nubmers of balls and strikes, the previous pitch, etc.)

In [ ]:
#importing some necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import pybaseball as pb
import numpy as np

In [ ]:
#getting the pitching information for all of Pressly's games for the 2022 season
raw_data = pb.statcast_pitcher('2022-04-07', '2022-10-05', player_id = 519151)
raw_data.info()

After looking at the different types of data available when getting the pitching information, I racked my brain to figure out what factors would be seemingly important to a pitcher in the short time frame they have between pitches. Obviously, the count (how many balls and strikes there are) is something considered by the pitcher. Additionally, the handedness of the batter (right or left handed) would change the type of pitch a pitcher tends to use. Maybe the pitch number for each at-bat will affect the pitcher's decision? One thing that I wasn't sure about was the last pitch and second to last pitch type; perhaps that might impact the pitcher's choice? Well, to test these theories, let's first do a bit of feature engineering and combine all the features into a singular, organized dataframe.

In [ ]:
#using the pitch type data to make the last pitch and second to last pitch features
last_pitch = raw_data.get(["pitch_name"]).shift(-1)
last_pitch = last_pitch.mask(last_pitch.eq('None')).dropna()
second_last = raw_data.get(["pitch_name"]).shift(-2)
second_last = second_last.mask(second_last.eq('None')).dropna()

In [ ]:
#creating the organized dataframe
df = raw_data.get(["game_date", "pitch_name", "pitch_number", "strikes", "balls", "stand"])
df.insert(3, "last_pitch", last_pitch)
df.insert(4, "second_last", second_last)

df

Alright, now that I have my organized dataframe of my features and labels, I need to encode all of it in order for it to, you know, actually work with the algorithms! First however, I removed all of the data that is the first or second pitch of the at-bat. My reasoning behind this is that at bats act as a sort of refresh for pitchers - a clean slate, if you will. Additionally, by shifting the data up by 1 and 2 for the last and second to last picth data respectively, for the first and second pitch of a game, the data would say that the last pitch and second last pitch would be from the last game, which just doesn't make sense! Finally, I'm going to combine two of the label types, Curveballs and Sliders, for two reasons. One, when grouping pitch types, people usually combine them as one category - Breaking Balls. The second reason is more important in terms of practicality; the pitches that are most important for batters to recognize (in order for them to hit it well) are fastballs and changeups, which means that differentiating between fastballs, changeups, and breaking balls is the most practical setup for this algorithm.

In [ ]:
#combining the curveballs and sliders

df = df.mask(df['pitch_number'].eq(1)).dropna()
df = df.mask(df['pitch_number'].eq(2)).dropna()

df['pitch_name'] = df['pitch_name'].replace('4-Seam Fastball', 'fast')
df['pitch_name'] = df['pitch_name'].replace(['Curveball', 'Slider'], 'breaking')


df['last_pitch'] = df['last_pitch'].replace('4-Seam Fastball', 'fast')
df['last_pitch'] = df['last_pitch'].replace(['Curveball', 'Slider'], 'breaking')

df['second_last'] = df['second_last'].replace('4-Seam Fastball', 'fast')
df['second_last'] = df['second_last'].replace(['Curveball', 'Slider'], 'breaking')

df

In [ ]:
#encoding yayyy woo hoo

df["pitch_name"] = df["pitch_name"].astype('category')
df["pitch_name"] = df["pitch_name"].cat.codes

df["last_pitch"] = df["last_pitch"].astype('category')
df["last_pitch"] = df["last_pitch"].cat.codes

df["second_last"] = df["second_last"].astype('category')
df["second_last"] = df["second_last"].cat.codes

df['strikes'] = df['strikes'].astype(int)
df['balls'] = df['balls'].astype(int)
df['pitch_number'] = df['pitch_number'].astype(int)

df['stand'] = df['stand'].astype('category')
df['stand'] = df['stand'].cat.codes

df = df.drop('game_date', 1)

df

In [ ]:
X = np.array(df.drop('pitch_name', 1))
y = np.array(df['pitch_name'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

In [ ]:
clf = RandomForestClassifier(n_estimators = 200)
clf.fit(X_train, y_train)

clf.score(X_test, y_test)

In [ ]:
rbf = svm.SVC(kernel='rbf', gamma=10, C=1).fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=2, C=0.13).fit(X_train, y_train)

In [ ]:
poly_pred = poly.predict(X_test)
rbf_pred = rbf.predict(X_test)

In [ ]:
poly_accuracy = accuracy_score(y_test, poly_pred)
poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Polynomial Kernel): ', "%.2f" % (poly_accuracy*100))
print('F1 (Polynomial Kernel): ', "%.2f" % (poly_f1*100))

rbf_accuracy = accuracy_score(y_test, rbf_pred)
rbf_f1 = f1_score(y_test, rbf_pred, average='weighted')
print('Accuracy (RBF Kernel): ', "%.2f" % (rbf_accuracy*100))
print('F1 (RBF Kernel): ', "%.2f" % (rbf_f1*100))